In [1]:
import tensorflow as tf

Going from one machine to many, might sound complicated, but as we will see, with the estimator API and ML engine managing the cluster automatically, you get distribution out of the box. The function that implements distributed training is called `tf.estimator.train_and_evaluate()`. The name of the function also highlights that evaluating and monitoring a large training job will be important. We will see that later. Let's focus for now on distribution.

# Distributed Training:

<img src="img/Snap_1.jpg" width="800" height="800"/>

<b>Data Parallelism: </b> Data Parallelism replicate your model on multiple workers.

The traditional distribution model for training neural networks is called data parallelism. Your model is replicated on multiple workers. At each training steps, these load a batch of training data, hopefully a different one each, compute their gradients and send them to one or several central parameter servers, which hold all the weights and biases of the neural network model. The gradients are applied as they arrive. Change the weights and biases, and the updated model is then sent back to workers for the next step of training.

There is a lot of work to do to make this happen. Workers must be started, then they receive their copy of the model, data flows between workers. Parameter servers must be established. The system must also handle exceptions and failures and if an incident occurs restart failed workers from where they left off. And check pointing also becomes a bit more complicated, when all this is going on.

Fortunately for you, distribution will be as simple as writing a config file. All the boilerplate code is already written in the estimator API(`tf.estimator.train_and_evaluate()`). You will need to do four things.

<li><b>Choose your estimator.</b></li>
<li><b>Provide a run configuration.</b></li>
<li><b>Provide training data through `train_spec`</b></li>
<li><b>Provide evaluating data through `eval_spec`</b></li>

Once that is set up, call `tf.estimator.train_and_evaluate()`. And if you are running on ML engine, and have specified the cluster size, distributed training will kick in.

<img src="img/Snap_2.jpg" width="800" height="800"/>

Lets have a look at above four things

# Run Configuration:

In [2]:
OUTPUT_DIR = "./trained_model"
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR, 
    save_summary_steps=100,
    save_checkpoints_steps=2000 # After every 2000 steps our model will save weights and biases.
)

The run config first. This is where you specify the output directory for checkpoints. You can still set it directly, when instantiating the estimator, but it's cleaner to have it here, along with other checkpoint settings. Indeed, this is also where you set the frequency at which you want to see checkpoints and also the frequency of your training logs or summaries.

# Estimator:

In [ ]:
estimator = tf.estimator.LinearRegressor(
    feature_columns=feat_cols,
    config=run_config)

# Train Spec

In [ ]:
train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, max_steps=50000)

The `tf.estimator.TrainSpec()`, is where you pass in your data input function for training data. Please use the data set API to set it up properly. Optionally, you can limit the training to a given number of steps. By default, training proceeds until the input data set is exhausted. Which might happen after multiple epochs if that's how you set it up.

# Eval Spec

In [ ]:
eval_spec = tf.estimator.EvalSpec(
    input_fn=eval_input_fn,
    steps=100, # We want to evaluate 100 batches of test data
    throttle_secs=600, # 10 Minutes (We want to run our evaluation on every 10 minutes)
    exporters=None
)

The `EvalSpec`, is where you plug in your test data set. Yes, if you want to see how well your model is doing, you have to measure that on a data set that it has not seen during training. Usually a subset of your data that you set aside for testing. The test data comes in through an `eval_input_fn` and again, please use the data set API to get it. 

You also get to specify on how many batches of test data you want to evaluate, and how frequently evaluations happen.  

One implementation detail to bear in mind, in distributed training, evaluation happens on a dedicated server, which respond the model from the latest checkpoint and then runs eval. So, you cannot get evaluations more frequently than the check points frequency you entered in your `run_config`. You can, however, get them less frequently, by adding the throttling parameter in the EvalSpec.

You notice that the EvalSpec also has a parameter for exporters. They control how a model is exported for deployment to production, and we will cover them in the next chapter.

# Running The Model:

In [ ]:
tf.estimator.train_and_evaluate(
    estimator=estimator,
    train_spec=train_spec,
    eval_spec=eval_spec
)

# Importance Of Data Shuffling In Distributed Training:

I want to say a couple of words about an important practical consideration, data shuffling. The stochastic gradient descent algorithm that neural networks use for training, only works on well-shuffled data. The data set API has a shuffle function that can help there, but some people might not use it if they think their data set is already well shuffled on disk. With distributed training, beware. Even with a well-shuffled data set on disk, if all your workers are loading straight from this data set, they will be seeing the same batch of data, at the same time, and produce the same gradients. The benefits of distributed training are wasted. Your multiple workers all do the exact same things.

In [ ]:
dataset = tf.data.Dataset.list_files("train.csv-*") \
                         .shuffle(100) \
                         .flat_mapla(tf.data.TextLineDataset) \
                         .map(decode_csv)

dataset = dataset.shuffle(1000) \
                 .repeat(15) \
                 .batch(128)

With data set that shuffle, the shuffling happens independently on each worker using a different random seed, so please use it. Even if your data comes already shuffled on disk. 

And if you want to be extra sure, you can also shuffle the list of filenames in your sharded data set. `tf.data.Dataset.list_files()`, returns a data set of filenames, so just call shuffle on it.